# CL Protocol Specification

Code is used to show the mathematics details for ZKP-CL.

The CL-signature scheme defines the public key in this key pair as the quadruple $(n, Z, S ,\{R_i\}_{i \in A})$ where $A$ is the set of indices of attributes in the credential schema, $n$ is a Special RSA Modulus, and $Z, S , {R_i}$ are random quadratic residues modulo n.

The public key information $(n, Z, S ,\{R_i\}_{i \in A})$ are publicly available on the ledger, the signature $(A, e, v)$ can only be calculated by the issuer, who owns the private key $(p, q)$.

## Equations


Remember from the math section:

$$ T = A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$
$$ A' = AS^r $$
$$ v' = v - er $$
$$ \hat{v} = \bar{v} + Dv'  $$
$$ e' = e - \beta $$
$$ \hat{e} = \bar{e} + De' $$
$$ \hat{m_i} = \bar{m_i} + Dm_i $$
$$ Q = A^e \pmod{n} $$
$$ Q = \Bigg(\frac{Z}{S^v \prod_{i \in A}{R_i^{m_i}}}\Bigg) \pmod{n} $$
$$ \hat{T} = \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n} $$

## Issuer setup

Let $i$ be the number of attributes we work with.

Steps to set up the issuer

1. Issuer chooses a number of attribute bases $i$
2. Generate random primes of equal length ($p$ and $q$) 
3. Then compute the modulus $n \leftarrow pq$.
4. Generate a random quadratic residue: $S \pmod{n}$
5. Generate quadratic residues $(Z,\{R_i\}_{i \in A}) \pmod{n}$ 

The issuer's public key is $(n, S, Z, R_1,R_2,\ldots,R_i)$ and the private key is $(p, q)$.

In [2]:
from Crypto.Util import number
import hashlib


In [3]:
# For demo purposes to make print more readable and consistent
bitsize = 32
p = 3397833353
q = 2516557159
# p = number.getPrime(bitsize)
# q = number.getPrime(bitsize)

# The public parts
n = p*q
nsize = 2*bitsize
# Generate quadratic residues mod n
R_1 = pow(number.getRandomRange(2, n), 2, n)
R_2 = pow(number.getRandomRange(2, n), 2, n)
R_3 = pow(number.getRandomRange(2, n), 2, n)
R_4 = pow(number.getRandomRange(2, n), 2, n)
R_5 = pow(number.getRandomRange(2, n), 2, n)
R_6 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)


print(f'Public key is:\nn = {n}\n(R_1, R_2, R_3, R_4, R_5, R_6) = {(R_1, R_2, R_3, R_4, R_5, R_6)}\nS = {S}\nZ = {Z}')
print(f'\nPrivate key is: {(p,q)}')

Public key is:
n = 8550841849581124127
(R_1, R_2, R_3, R_4, R_5, R_6) = (7489092437948868914, 4638538772027415986, 6123285392668868892, 7869184347826771002, 4040922565690684154, 452834554926598509)
S = 3474591476129189134
Z = 6979000837685429326

Private key is: (3397833353, 2516557159)


In [4]:
# The messages
msize = nsize
m_1 = number.getRandomRange(2, n)
m_2 = number.getRandomRange(2, n)
m_3 = number.getRandomRange(2, n)
m_4 = number.getRandomRange(2, n)
m_5 = number.getRandomRange(2, n)
m_6 = number.getRandomRange(2, n)

print(f'Message is:\nm_1, m_2, m_3, m_4, m_5, m_6)  = {(m_1, m_2, m_3, m_4, m_5, m_6)}')

Message is:
m_1, m_2, m_3, m_4, m_5, m_6)  = (7273425162531509682, 4341954895574213369, 6590368354271484869, 6365936663992550230, 2836189369454369458, 1223581943782845983)


## User

User loads Issuer's public key and calculate:

1. Generates random integer $v'$
2. Then computes
$$U \leftarrow S^{v'}R_1^{m_1}\pmod{n}$$

In [5]:
# Generate random integear
v_prim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate U
U = pow(S, v_prim, n) * pow(R_1, m_1, n)

### User send to Issuer
User send $(U, v')$ to the Issuer

## Issuer Signature Generation


Issuer generate CL signature on the attributes by computing:

1. Generates random integer $v''$
2. Generates random prime $e$
3. Then computes 
$$
Q \leftarrow \frac{Z}{U S^{v''}(R_2^{m_2}R_3^{m_3}\cdots  R_i^{m_i})}\pmod{n}.
$$
4. Then computes
$$
A \leftarrow Q^{e^{-1}\pmod{p'q'}}\pmod{n}.
$$


In [6]:
# Generate random prime
e = number.getPrime(nsize + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
# v is selected as integer
v_pprim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate dot
R_vector = pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_4, m_4, n) * pow(R_5, m_5, n) * pow(R_6, m_6, n)

# Calculate commitment
commitment_vector = U * pow(S, v_pprim, n) * R_vector
commitment_vector_inv = pow(commitment_vector, -1, n)

# Calculate Q
Q = (Z * commitment_vector_inv) % n
# Calculate signature part A
A = pow(Q, e_inv, n)

print(f"Pre-signature is: A = {A}, e = {e}, v'' = {v_pprim}")

Pre-signature is: A = 1796123067940129401, e = 35181504651618055519, v'' = 1906501872935750363047670476252607561710659671527057754067312043075675421785668747974394696839296131836108839005446823930095874734199783894950880592184613163079422424746780125609641866819890434383394800225512769599449002778327445255992304416943239601893219438030124486571115111562532842496202930769352640934881371696812046605766148148619909708174861577870960724054097721546750306589633437049685388808707700207033307578485949083789778951468415507154303725717221679546961674587269001452711421451889351699432156419089714304005891825329419609499842892547430910623904688271134366646569646393189608692807870095291574027518668073570717055488968248230987496641067905761803260491393142621754815499178732408867763637375412822919613559609980084195745296661532258561830256352360531169999639108117328703098186136091308989703513853048849108290054170571424323775895921333416009091104930622628268175190826680631643490232227582867825461150083

Check
$$ Q = A^e $$

In [7]:
# verify
print('RHS is the same as LHS:', pow(A, e, n) == Q)

RHS is the same as LHS: True


### Issuer sends to User
Issuer sends pre-signature $(A,e,v'')$ to the User

## User complete signature

For signature completion user computes
$$v \leftarrow v'+v''$$

In [8]:
v = v_prim + v_pprim

User now know the signature $(A,e,v)$

User verify the signature $(A,e,v)$, using the CL verification algorithm:
1. Compute
$$
\hat{Z} \leftarrow A^{e} S^{v}(R_1^{m_1}R_2^{m_2}\cdots  R_i^{m_i})\pmod{n}.
$$

In [9]:
R_dot = pow(R_1, m_1, n) * R_vector
Z_hat = (pow(A, e, n) * pow(S, v, n) * R_dot) % n

Check
$$ Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) $$

In [10]:
print('RHS is the same as LHS:', Z_hat == Z)

RHS is the same as LHS: True


## Proof Generation


### Selective disclosure
Let A be the set of all attribute identifiers present in the credential schema
such that $A_r$ contains revealed attributes to the Verifier and $A_h$ contains unrevealed (hidden) attributes which are kept secret.


In [11]:
# public attributes
R_public = pow(R_4, m_4, n) * pow(R_5, m_5, n)
# private attributes
R_private = pow(R_1, m_1, n) * pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_6, m_6, n)

We remember that:
$$ Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) = 
A^e S^v \Bigg(\prod_{i \in A_r}{R_i^{m_i}} \cdot \prod_{i \in A_h}{R_i^{m_i}}\Bigg)
$$

Check
$$ Z \Bigg(\prod_{i \in A_r}{R_i^{m_i}}\Bigg)^{-1} = A^e S^v \Bigg(\prod_{i \in A_h}{R_i^{m_i}}\Bigg) $$

In [12]:
# generate commitment and encryption
R_public_inv = pow(R_public, -1, n)
# contains revealed attributes
attr_encryption_vector = Z * R_public_inv

# contains unrevealed attributes which are kept secret
attr_commitment_vector = pow(S, v, n) * R_private

# verify credentials
print('RHS is the same as LHS:', (pow(A, e, n) * attr_commitment_vector) % n == attr_encryption_vector % n)

RHS is the same as LHS: True


For each non-revealed attribute 

1. Generate random integer $\bar{m_1}, \bar{m_2}, \bar{m_3}$ for the secret attributes
2. Generates random integer $\bar{m_j}$ for each non-revealed attribute


In [13]:
m1_bar = number.getRandomNBitInteger(nsize)
m2_bar = number.getRandomNBitInteger(nsize)
m3_bar = number.getRandomNBitInteger(nsize)
m6_bar = number.getRandomNBitInteger(nsize)


Rbar_private = pow(R_1, m1_bar, n) * pow(R_2, m2_bar, n) * pow(R_3, m3_bar, n) * pow(R_6, m6_bar, n)


1. Generate random integer $r$
2. Then computes 
$$
A' \leftarrow A S^{r}\pmod{n}
$$
4. Then computes
$$
v' \leftarrow v - e\cdot r
$$


In [14]:
# Choose random bit r
r = number.getRandomNBitInteger(nsize + 1)

# Calculate
A_prime = A * pow(S, r, n)
v_prime = v - e*r

We remember that:

$$
Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) = A'^{e} S^{v'} \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) 
$$

Check
$$
A^e S^v = A'^{e} S^{v'}
$$
$$
A' S^{-r} = A \pmod{n}
$$

In [15]:
# we know that

print('RHS is the same as LHS:', (pow(A_prime, e, n) * pow(S, v_prime, n) % n * R_dot) % n == Z)
print('RHS is the same as LHS:', (pow(A_prime, e, n) * pow(S, v_prime, n) % n == pow(A, e, n) * pow(S, v, n) % n))
print('RHS is the same as LHS:', ((A_prime * pow(S, -r, n) % n == A % n)))

RHS is the same as LHS: True
RHS is the same as LHS: True
RHS is the same as LHS: True


1. Generate random integer $\beta$
2. Compute 
$$e' \leftarrow e - \beta$$

In [16]:
beta = number.getRandomNBitInteger(bitsize + 1)
e_prime = e - beta

print(f"Randomized signature is:\nA = {A_prime},\ne' = {e_prime},\nv' = {v_prime}")




Randomized signature is:
A = 3331733577123776045443201618297596630,
e' = 35181504645426043147,
v' = 296489540370227338957468148067940772404618942059488994386726627179044680653843875618225968698895111591978659974829382064665390025917503573819626845383901361081497752264266222875498309588377921887934562897674281666322195151889402253345649337774007262146403236996643698816327728575296727255567834070867040665765004268612896011326193316302928750320085762541942878007869075274468340085781640243471336543516930115315787484993093779025660761330739115229253383047738945170871527161892321875606713041850583476078062380390353860862363230785092358870447667759678606195713179119619545928420879024270405888134140497545618473870976337092177682131475876585431038860562944396151616674561462809947838106775550987700073044052203896376466585173069874159286163689519751575956360064068818378987724742230246975605930858765241915544600316139483083222562002852185603727684042452043756725799781621238258770461710863788726502

1. Generate random integer number $\bar{e}$
2. Generate random integer number $\bar{v}$
3. Compute
$$ T \leftarrow A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$

In [17]:
e_bar = number.getRandomNBitInteger(nsize + 1)
v_bar = number.getRandomNBitInteger(msize*nsize+1)
T = (pow(A_prime, e_bar, n) * pow(S, v_bar, n) * Rbar_private) % n

### Hashing

Compute 
$$
c\leftarrow H(\mathcal{T},\mathcal{C},n_1)
$$

In [18]:
attr = str([T, Q, A, e, v, A_prime])
c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
c = int(c_hash, base=32)
print(f"Proof is: T = {T}, \nc = {c}")

Proof is: T = 4443983947906183329, 
c = 953339395777708075496704772466219349584164381227043364721344626679724506545530876457483130382761


Compute
\begin{align*}
\hat{v} \leftarrow \bar{v} + Dv' \\
\hat{e} \leftarrow \bar{e} + De'
\end{align*}

In [19]:
e_hat = e_bar + c*e_prime
v_hat = v_bar + c*v_prime

For each none revealed attribute, $j\in A_{h}$, compute
$$ \hat{m_j} \leftarrow \bar{m_j} + Dm_j $$


In [20]:
m1_hat = m1_bar + c*m_1
m2_hat = m2_bar + c*m_2
m3_hat = m3_bar + c*m_3
m6_hat = m6_bar + c*m_6

Rhat_private = pow(R_1, m1_hat, n) * pow(R_2, m2_hat, n) * pow(R_3, m3_hat, n) * pow(R_6, m6_hat, n)

We remember that:

\begin{equation}

\Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \\

= {\Bigg(\prod_{i \in A_r}{R_i^{Dm_i}}} \cdot \prod_{i \in A_h}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \\

= \Bigg(\prod_{i \in A}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \pmod{n} \\
\end{equation}

Lets check

\begin{equation}

\Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \\

= \Bigg(\prod_{i \in A}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \pmod{n} \\
\end{equation}


In [21]:
Rc_public = pow(R_4, m_4*c, n) * pow(R_5, m_5*c, n)
Rc_private = pow(R_1, m_1*c, n) * pow(R_2, m_2*c, n) * pow(R_3, m_3*c, n) * pow(R_6, m_6*c, n)
Rc_dot = Rc_private * Rc_public

print('RHS is the same as LHS:', (Rc_public * Rhat_private % n == Rc_dot * Rbar_private % n))

RHS is the same as LHS: True


## Proof Verification

Calculate

$$
\hat{T} \leftarrow \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg)
= Z^{-D} \cdot A'^{D\beta} \cdot A'^{\hat{e}} \cdot S^{\hat{v}} \cdot \Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n}
$$

In [22]:
T_hat = (pow(Z, -c, n) * pow(A_prime, c*beta, n) * pow(A_prime, e_hat, n) * pow(S, v_hat, n) * Rc_public * Rhat_private) % n

# we know that
print('RHS is the same as LHS:', (T_hat) == T)

RHS is the same as LHS: True


Compute
$$
\hat{Q} \leftarrow A'^{e} S^{-er} \pmod{n}
$$

In [23]:
Q_hat = pow(A_prime, e, n) * pow(S, -e*r, n) % n

Compute 
$$
\widehat{c} \leftarrow H(\hat{\mathcal{T}},\mathcal{C},n_1);
$$

In [24]:


attr_hat = str([T_hat, Q_hat, A, e, v, A_prime])
c_hash_hat = hashlib.sha256(str(attr_hat).encode('utf-8')).hexdigest()
c_hat = int(c_hash_hat, base=32)

print('RHS is the same as LHS:', (c_hat) == c)

RHS is the same as LHS: True


## Equality proof
This zero knowledge proof is in fact just an equality proof that does not reveal the attribute’s value.

In [25]:
assert([T, c] == [T_hat, c_hat])

In [27]:
print('The equality has been proven when this line is printed!')

The equality has been proven when this line is printed!
